<p style="color:red;">Name: Parham Rezaei</P>
<p style="color:green;">STD Number: 400108547</p>

colab stuff

In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-s23h9h2c
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-s23h9h2c
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4294 sha256=25f6e5c75abdac596c71049874c4e1165d6a24a514a044e6b20db04e66e4fa5d
  Stored in directory: /tmp/pip-ephem-wheel-cache-eic3807l/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


# Problem 1 (بخش الف)

I implemented it using C for matrix size 1024*1024.
first I allocated the space for the pointers and the array itself then I filled it randomly with float numbers.</b>
Afterwards, I called the multiply function which performs the multiplication algorithm by iterating through all rows and collumns. Lastly, I freed the used memory.

In [42]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

void multiply(float **p1, float **p2, float **p3, int m);
int main(){
    int m = 1024;
    int fpsize = m * sizeof(float *);
    int fsize = m * sizeof(float);
    // first matrix
    float **p1 = (float **)malloc(fpsize);
    // second matrix
    float **p2 = (float **)malloc(fpsize);
    // result matrix
    float **p3 = (float **)malloc(fpsize);
    // allocating each row of the matrix
    for (int i = 0; i < m; i++){
        p1[i] = (float *)malloc(fsize);
        p2[i] = (float *)malloc(fsize);
        p3[i] = (float *)calloc(m,sizeof(float)); // calloc so no need to initialize to 0
    }
    // fill matrices randomly
    for(int i = 0; i < m; i++){
        for(int j = 0; j < m; j++){
            p1[i][j] = (float)rand() / RAND_MAX;
            p2[i][j] = (float)rand() / RAND_MAX;
        }
    }
    // calculating the time
    clock_t start,end;
    start = clock();
    // doing the multiplication
    multiply(p1, p2, p3, m);
    end = clock();
    // calculate the time it took
    double time = ((double)(end-start))/CLOCKS_PER_SEC;
    printf("CPU Matrix Multiplication Size %d took %f seconds\n", m, time);

    // free the allocated memory
    for(int i = 0; i < m; i++){
        free(p1[i]);
        free(p2[i]);
        free(p3[i]);
    }
    free(p1);
    free(p2);
    free(p3);
    return 0;
}


void multiply(float **p1, float **p2, float **p3, int m){
    for(int i = 0; i < m; i++){
        for(int j = 0; j < m; j++){
            for(int l = 0; l < m; l++){
                p3[i][j] += p1[i][l] * p2[l][j];
            }
        }
    }
}

CPU Matrix Multiplication Size 1024 took 9.248623 seconds



we see that it takes more than any other code in this hw

# Problem 2 (بخش ب)

First of all I allocate the cpu memory as before. I use malloc for cuda too (with cudaMalloc) and allocate the same space there too which is stored in variable size. I also fill the matrices randomly and copy them to device from host.<br>
Then, I call the multiply kernel which first finds the corresponding row and column for the thread and if it is in the bound iterates through memory values and calculates the result.<br>
I also have a cpu function that checks the results after copying back to host with a simple accurate cpu algorithm to make sure the results are within a correct interval.

In [44]:
%%cu
#include <stdio.h>
#include <stdlib.h>

__global__ void multiply(const float *a, const float *b, float *c, int m){
    // defining row and column based on current thread
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if(row < m && col < m){ // check for being inside the bounds
        float sum = 0;
        for (int i=0; i<m; i++){
            sum+= a[row*m+i] * b[i*m+col]; //dot product
        }
        c[row*m+col] = sum; //assign the value to the result
    }
}

void checkMultiplicationCorrect(float *p1, float *p2, float *p3, int m){
    for (int i=0; i<m; i++){
        for (int j=0; j<m; j++){
            float val = 0;
            for (int k=0; k<m; k++){
                val += p1[i*m+k] * p2[k*m+j];
            }
            // check for being close enough
            if (abs(p3[i*m+j] - val) > 0.0001){
                printf("\nwrong multiplication\n");
                return;
            }
        }
    }
    printf("\ncorrect multiplication\n");
}


int main(){
    int m = 1024;
    size_t size = m * m * sizeof(float);

    // allocate memory for host
    float *p1 = (float *)malloc(size);
    float *p2 = (float *)malloc(size);
    float *p3 = (float *)malloc(size);

    // initialize matrices
    for (int i=0; i<m*m; i++){
        p1[i] = (float)rand()/RAND_MAX;
        p2[i] = (float)rand()/RAND_MAX;
    }
    // allocate memory for device
    float *dp1, *dp2, *dp3;
    cudaMalloc(&dp1, size);
    cudaMalloc(&dp2, size);
    cudaMalloc(&dp3, size);

    // copy data to device from host
    cudaMemcpy(dp1, p1, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dp2, p2, size, cudaMemcpyHostToDevice);

    // launch kernel
    int thr = 32; //threads in each axis
    int blc = m / thr; //blocks to make all matrix size
    dim3 threads = dim3(thr, thr);
    dim3 blocks = dim3(blc, blc);
    // get the time of multiplication
    cudaEvent_t begin,end;
    cudaEventCreate(&begin);
    cudaEventCreate(&end);
    cudaEventRecord(begin);
    // call kernel
    multiply<<<blocks, threads>>>(dp1, dp2, dp3, m);

    cudaEventRecord(end);
    cudaEventSynchronize(end);
    float time = 0;
    cudaEventElapsedTime(&time, begin, end);
    // copy back
    cudaMemcpy(p3, dp3, size, cudaMemcpyDeviceToHost);

    printf("Matrix Multiplication GPU Size %d takes %f seconds", m,time/1000);
    checkMultiplicationCorrect(p1, p2, p3, m);
    //free memory
    cudaFree(dp1);
    cudaFree(dp2);
    cudaFree(dp3);

    free(p1);
    free(p2);
    free(p3);
}

Matrix Multiplication GPU Size 1024 takes 0.006643 seconds
correct multiplication



> much less than the cpu one

here I implement the 10240 one, but won't check the result becasue of getting too long on cpu

In [46]:
%%cu
#include <stdio.h>
#include <stdlib.h>

__global__ void multiply(const float *a, const float *b, float *c, int m){
    // defining row and column based on current thread
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if(row < m && col < m){ // check for being inside the bounds
        float sum = 0;
        for (int i=0; i<m; i++){
            sum+= a[row*m+i] * b[i*m+col]; //dot product
        }
        c[row*m+col] = sum; //assign the value to the result
    }
}

void checkMultiplicationCorrect(float *p1, float *p2, float *p3, int m){
    for (int i=0; i<m; i++){
        for (int j=0; j<m; j++){
            float val = 0;
            for (int k=0; k<m; k++){
                val += p1[i*m+k] * p2[k*m+j];
            }
            // check for being close enough
            if (abs(p3[i*m+j] - val) > 0.0001){
                printf("\nwrong multiplication\n");
                return;
            }
        }
    }
    printf("\ncorrect multiplication\n");
}


int main(){
    int m = 10240;
    size_t size = m * m * sizeof(float);

    // allocate memory for host
    float *p1 = (float *)malloc(size);
    float *p2 = (float *)malloc(size);
    float *p3 = (float *)malloc(size);

    // initialize matrices
    for (int i=0; i<m*m; i++){
        p1[i] = (float)rand()/RAND_MAX;
        p2[i] = (float)rand()/RAND_MAX;
    }
    // allocate memory for device
    float *dp1, *dp2, *dp3;
    cudaMalloc(&dp1, size);
    cudaMalloc(&dp2, size);
    cudaMalloc(&dp3, size);

    // copy data to device from host
    cudaMemcpy(dp1, p1, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dp2, p2, size, cudaMemcpyHostToDevice);

    // launch kernel
    int thr = 32; //threads in each axis
    int blc = m / thr; //blocks to make all matrix size
    dim3 threads = dim3(thr, thr);
    dim3 blocks = dim3(blc, blc);
    // get the time of multiplication
    cudaEvent_t begin,end;
    cudaEventCreate(&begin);
    cudaEventCreate(&end);
    cudaEventRecord(begin);
    // call kernel
    multiply<<<blocks, threads>>>(dp1, dp2, dp3, m);

    cudaEventRecord(end);
    cudaEventSynchronize(end);
    float time = 0;
    cudaEventElapsedTime(&time, begin, end);
    // copy back
    cudaMemcpy(p3, dp3, size, cudaMemcpyDeviceToHost);

    printf("Matrix Multiplication GPU Size %d takes %f seconds", m,time/1000);

    //free memory
    cudaFree(dp1);
    cudaFree(dp2);
    cudaFree(dp3);

    free(p1);
    free(p2);
    free(p3);
}

Matrix Multiplication GPU Size 10240 takes 3.765575 seconds


> clearly more than the 1024 size. Still way faster than cpu

# Problem 3 (بخش ج)

- This one was removed from the hw (announced in Telegram)

Like before, I implemented the process of using the kernel. Here I have two constant values of trsize and matsize. They are because of using the shared memory as it required me to use constant values.<br>
we first load onto the shared memory and then calculate based on the indices of this memory. Access to shared memory which is common for a block is much quicker than accessing the global memory each time. Notice we still need to use the global memory while filling the shared memory.

here is a one without sharing, I use 4 threads to make the results suitable for comparing.

In [48]:
%%cu
#include <stdio.h>
#include <stdlib.h>

__global__ void multiply(const float *a, const float *b, float *c, int m){
    // defining row and column based on current thread
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if(row < m && col < m){ // check for being inside the bounds
        float sum = 0;
        for (int i=0; i<m; i++){
            sum+= a[row*m+i] * b[i*m+col]; //dot product
        }
        c[row*m+col] = sum; //assign the value to the result
    }
}

void checkMultiplicationCorrect(float *p1, float *p2, float *p3, int m){
    for (int i=0; i<m; i++){
        for (int j=0; j<m; j++){
            float val = 0;
            for (int k=0; k<m; k++){
                val += p1[i*m+k] * p2[k*m+j];
            }
            // check for being close enough
            if (abs(p3[i*m+j] - val) > 0.0001){
                printf("\nwrong multiplication\n");
                return;
            }
        }
    }
    printf("\ncorrect multiplication\n");
}


int main(){
    int m = 1024;
    size_t size = m * m * sizeof(float);

    // allocate memory for host
    float *p1 = (float *)malloc(size);
    float *p2 = (float *)malloc(size);
    float *p3 = (float *)malloc(size);

    // initialize matrices
    for (int i=0; i<m*m; i++){
        p1[i] = (float)rand()/RAND_MAX;
        p2[i] = (float)rand()/RAND_MAX;
    }
    // allocate memory for device
    float *dp1, *dp2, *dp3;
    cudaMalloc(&dp1, size);
    cudaMalloc(&dp2, size);
    cudaMalloc(&dp3, size);

    // copy data to device from host
    cudaMemcpy(dp1, p1, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dp2, p2, size, cudaMemcpyHostToDevice);

    // launch kernel
    int thr = 4; //threads in each axis
    int blc = m / thr; //blocks to make all matrix size
    dim3 threads = dim3(thr, thr);
    dim3 blocks = dim3(blc, blc);
    // get the time of multiplication
    cudaEvent_t begin,end;
    cudaEventCreate(&begin);
    cudaEventCreate(&end);
    cudaEventRecord(begin);
    // call kernel
    multiply<<<blocks, threads>>>(dp1, dp2, dp3, m);

    cudaEventRecord(end);
    cudaEventSynchronize(end);
    float time = 0;
    cudaEventElapsedTime(&time, begin, end);
    // copy back
    cudaMemcpy(p3, dp3, size, cudaMemcpyDeviceToHost);

    printf("Matrix Multiplication GPU Size %d takes %f seconds", m,time/1000);
    checkMultiplicationCorrect(p1, p2, p3, m);
    //free memory
    cudaFree(dp1);
    cudaFree(dp2);
    cudaFree(dp3);

    free(p1);
    free(p2);
    free(p3);
}

Matrix Multiplication GPU Size 1024 takes 0.028759 seconds
correct multiplication



shared

In [50]:
%%cu
#include <stdio.h>
#include <stdlib.h>

// the values in __shared__ should be constant so I define them here
#define trsize 4
#define matsize 1024
//CUDA kernel
__global__ void multiply(const float *a, const float *b, float *c, int m){
    // row and column index for current thread
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    // shared memory
    __shared__ float s_a[trsize][matsize];
    __shared__ float s_b[matsize][trsize];

    //load data to shared memory
    for (int i=0; i<m/trsize; i++){
        s_a[threadIdx.y][threadIdx.x + i*trsize] = a[row*m + threadIdx.x + i*trsize];
        s_b[threadIdx.y + i*trsize][threadIdx.x] = b[(threadIdx.y + i*trsize)*m + col];
    }
    // make sure all data is loaded
    __syncthreads();

    float tmp = 0.0f;
    for (int k=0;k<m;k++){
        tmp += s_a[threadIdx.y][k] * s_b[k][threadIdx.x]; // dot product
    }
    if (col < m && row < m) c[m*row + col] = tmp; // assign to result
}

void checkMultiplicationCorrect(float *p1, float *p2, float *p3, int m){
    for (int i=0; i<m; i++){
        for (int j=0; j<m; j++){
            float val = 0;
            for (int k=0; k<m; k++){
                val += p1[i*m+k] * p2[k*m+j];
            }
            // check for being close enough
            if (abs(p3[i*m+j] - val) > 0.0001){
                printf("\nwrong multiplication\n");
                return;
            }
        }
    }
    printf("\ncorrect multiplication\n");
}


int main(){
    int m = matsize;
    size_t size = m * m * sizeof(float);

    // allocate memory for host
    float *p1 = (float *)malloc(size);
    float *p2 = (float *)malloc(size);
    float *p3 = (float *)malloc(size);

    // initialize matrices
    for (int i=0; i<m*m; i++){
        p1[i] = (float)rand()/RAND_MAX;
        p2[i] = (float)rand()/RAND_MAX;
    }
    // allocate memory for device
    float *dp1, *dp2, *dp3;
    cudaMalloc(&dp1, size);
    cudaMalloc(&dp2, size);
    cudaMalloc(&dp3, size);

    // copy data to device from host
    cudaMemcpy(dp1, p1, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dp2, p2, size, cudaMemcpyHostToDevice);

    // launch kernel
    int thr = trsize; //threads in each axis
    int blc = m / thr; //blocks to make all matrix size
    dim3 threads = dim3(thr, thr);
    dim3 blocks = dim3(blc, blc);
    // get the time of multiplication
    cudaEvent_t begin,end;
    cudaEventCreate(&begin);
    cudaEventCreate(&end);
    cudaEventRecord(begin);
    // call kernel
    multiply<<<blocks, threads>>>(dp1, dp2, dp3, m);

    cudaEventRecord(end);
    cudaEventSynchronize(end);
    float time = 0;
    cudaEventElapsedTime(&time, begin, end);
    // copy back
    cudaMemcpy(p3, dp3, size, cudaMemcpyDeviceToHost);

    printf("Matrix Multiplication GPU Size %d takes %f seconds", m,time/1000);
    checkMultiplicationCorrect(p1, p2, p3, m);
    //free memory
    cudaFree(dp1);
    cudaFree(dp2);
    cudaFree(dp3);
    free(p1);
    free(p2);
    free(p3);
}

Matrix Multiplication GPU Size 1024 takes 0.048847 seconds
correct multiplication



# Problem 4 (بخش د)

Here is use tiles. I divide the matrix into several tiles and calculate the result on each of them separately. Using this sort of synchronize accesses to the shared memory in the phases caused by using tiles we see a bit of improvement. Also we have an edge by using tiles as they reduce the on-chip memory.

The implementation is similar as before. The main difference is in the kernel which we iterate through the tiles and load from memory and calculate result partially on each iteration.

1024

In [52]:
%%cu
#include <stdio.h>
#include <stdlib.h>

#define matsize 1024
#define tsize 4
//CUDA kernel
__global__ void multiply(const float *a, const float *b, float *c, int m){
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    float sum = 0.0f;
    __shared__ float s_a[tsize][tsize];
    __shared__ float s_b[tsize][tsize];

    for(int i=0;i < m/tsize; i++){
        s_a[threadIdx.y][threadIdx.x] = a[row*m + threadIdx.x + i*tsize];
        s_b[threadIdx.y][threadIdx.x] = b[m*(threadIdx.y+i*tsize)+col];
        __syncthreads();
        for(int p=0;p<tsize;p++){
            sum += s_a[threadIdx.y][p]*s_b[p][threadIdx.x];
        }
        __syncthreads();
    }
    if (col < m && row < m) c[m*row + col] = sum;
}

void checkMultiplicationCorrect(float *p1, float *p2, float *p3, int m){
    for (int i=0; i<m; i++){
        for (int j=0; j<m; j++){
            float val = 0;
            for (int k=0; k<m; k++){
                val += p1[i*m+k] * p2[k*m+j];
            }
            // check for being close enough
            if (abs(p3[i*m+j] - val) > 0.0001){
                printf("\nwrong multiplication\n");
                return;
            }
        }
    }
    printf("\ncorrect multiplication\n");
}


int main(){
    int m = matsize;
    size_t size = m * m * sizeof(float);

    //allocate memory for host
    float *p1 = (float *)malloc(size);
    float *p2 = (float *)malloc(size);
    float *p3 = (float *)malloc(size);

    //initialize host memory
    for (int i=0; i<m*m; i++){
        p1[i] = (float)rand()/RAND_MAX;
        p2[i] = (float)rand()/RAND_MAX;
    }
    float *dev1, *dev2, *dev3;
    cudaMalloc(&dev1, size);
    cudaMalloc(&dev2, size);
    cudaMalloc(&dev3, size);

    // copy data to device from host
    cudaMemcpy(dev1, p1, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dev2, p2, size, cudaMemcpyHostToDevice);

    //launch kernel
    int thr = tsize;
    int blc = m / thr;
    dim3 threads = dim3(thr, thr);
    dim3 blocks = dim3(blc, blc);
    // get the time of multiplication
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);
    multiply<<<blocks, threads>>>(dev1, dev2, dev3, m);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float time = 0;
    cudaEventElapsedTime(&time, start, stop);
    // copy back
    cudaMemcpy(p3, dev3, size, cudaMemcpyDeviceToHost);

    printf("time of multiplication with gpu: %f\n", time/1000);
    checkMultiplicationCorrect(p1, p2, p3, m);
    //free memory
    cudaFree(dev1);
    cudaFree(dev2);
    cudaFree(dev3);
    free(p1);
    free(p2);
    free(p3);
}

time of multiplication with gpu: 0.030304

correct multiplication



testing larger thread size. still 1024 matrix size

In [54]:
%%cu
#include <stdio.h>
#include <stdlib.h>

#define matsize 1024
#define tsize 32
//CUDA kernel
__global__ void multiply(const float *a, const float *b, float *c, int m){
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    float sum = 0.0f;
    __shared__ float s_a[tsize][tsize];
    __shared__ float s_b[tsize][tsize];

    for(int i=0;i < m/tsize; i++){
        s_a[threadIdx.y][threadIdx.x] = a[row*m + threadIdx.x + i*tsize];
        s_b[threadIdx.y][threadIdx.x] = b[m*(threadIdx.y+i*tsize)+col];
        __syncthreads();
        for(int p=0;p<tsize;p++){
            sum += s_a[threadIdx.y][p]*s_b[p][threadIdx.x];
        }
        __syncthreads();
    }
    if (col < m && row < m) c[m*row + col] = sum;
}

void checkMultiplicationCorrect(float *p1, float *p2, float *p3, int m){
    for (int i=0; i<m; i++){
        for (int j=0; j<m; j++){
            float val = 0;
            for (int k=0; k<m; k++){
                val += p1[i*m+k] * p2[k*m+j];
            }
            // check for being close enough
            if (abs(p3[i*m+j] - val) > 0.0001){
                printf("\nwrong multiplication\n");
                return;
            }
        }
    }
    printf("\ncorrect multiplication\n");
}


int main(){
    int m = matsize;
    size_t size = m * m * sizeof(float);

    //allocate memory for host
    float *p1 = (float *)malloc(size);
    float *p2 = (float *)malloc(size);
    float *p3 = (float *)malloc(size);

    //initialize host memory
    for (int i=0; i<m*m; i++){
        p1[i] = (float)rand()/RAND_MAX;
        p2[i] = (float)rand()/RAND_MAX;
    }
    float *dev1, *dev2, *dev3;
    cudaMalloc(&dev1, size);
    cudaMalloc(&dev2, size);
    cudaMalloc(&dev3, size);

    // copy data to device from host
    cudaMemcpy(dev1, p1, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dev2, p2, size, cudaMemcpyHostToDevice);

    //launch kernel
    int thr = tsize;
    int blc = m / thr;
    dim3 threads = dim3(thr, thr);
    dim3 blocks = dim3(blc, blc);
    // get the time of multiplication
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);
    multiply<<<blocks, threads>>>(dev1, dev2, dev3, m);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float time = 0;
    cudaEventElapsedTime(&time, start, stop);
    // copy back
    cudaMemcpy(p3, dev3, size, cudaMemcpyDeviceToHost);

    printf("time of multiplication with gpu: %f\n", time/1000);
    // checkMultiplicationCorrect(p1, p2, p3, m);
    //free memory
    cudaFree(dev1);
    cudaFree(dev2);
    cudaFree(dev3);
    free(p1);
    free(p2);
    free(p3);
}

time of multiplication with gpu: 0.005266



we see much improvement in comparison to previous methods

10240 without checking the result

In [56]:
%%cu
#include <stdio.h>
#include <stdlib.h>

#define matsize 10240
#define tsize 4
//CUDA kernel
__global__ void multiply(const float *a, const float *b, float *c, int m){
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    float sum = 0.0f;
    __shared__ float s_a[tsize][tsize];
    __shared__ float s_b[tsize][tsize];

    for(int i=0;i < m/tsize; i++){
        s_a[threadIdx.y][threadIdx.x] = a[row*m + threadIdx.x + i*tsize];
        s_b[threadIdx.y][threadIdx.x] = b[m*(threadIdx.y+i*tsize)+col];
        __syncthreads();
        for(int p=0;p<tsize;p++){
            sum += s_a[threadIdx.y][p]*s_b[p][threadIdx.x];
        }
        __syncthreads();
    }
    if (col < m && row < m) c[m*row + col] = sum;
}

void checkMultiplicationCorrect(float *p1, float *p2, float *p3, int m){
    for (int i=0; i<m; i++){
        for (int j=0; j<m; j++){
            float val = 0;
            for (int k=0; k<m; k++){
                val += p1[i*m+k] * p2[k*m+j];
            }
            // check for being close enough
            if (abs(p3[i*m+j] - val) > 0.0001){
                printf("\nwrong multiplication\n");
                return;
            }
        }
    }
    printf("\ncorrect multiplication\n");
}


int main(){
    int m = matsize;
    size_t size = m * m * sizeof(float);

    //allocate memory for host
    float *p1 = (float *)malloc(size);
    float *p2 = (float *)malloc(size);
    float *p3 = (float *)malloc(size);

    //initialize host memory
    for (int i=0; i<m*m; i++){
        p1[i] = (float)rand()/RAND_MAX;
        p2[i] = (float)rand()/RAND_MAX;
    }
    float *dev1, *dev2, *dev3;
    cudaMalloc(&dev1, size);
    cudaMalloc(&dev2, size);
    cudaMalloc(&dev3, size);

    // copy data to device from host
    cudaMemcpy(dev1, p1, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dev2, p2, size, cudaMemcpyHostToDevice);

    //launch kernel
    int thr = tsize;
    int blc = m / thr;
    dim3 threads = dim3(thr, thr);
    dim3 blocks = dim3(blc, blc);
    // get the time of multiplication
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);
    multiply<<<blocks, threads>>>(dev1, dev2, dev3, m);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float time = 0;
    cudaEventElapsedTime(&time, start, stop);
    // copy back
    cudaMemcpy(p3, dev3, size, cudaMemcpyDeviceToHost);

    printf("time of multiplication with gpu: %f\n", time/1000);
    // checkMultiplicationCorrect(p1, p2, p3, m);
    //free memory
    cudaFree(dev1);
    cudaFree(dev2);
    cudaFree(dev3);
    free(p1);
    free(p2);
    free(p3);
}

time of multiplication with gpu: 16.387831



making it faster by using more thread size

In [58]:
%%cu
#include <stdio.h>
#include <stdlib.h>

#define matsize 10240
#define tsize 32
//CUDA kernel
__global__ void multiply(const float *a, const float *b, float *c, int m){
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    float sum = 0.0f;
    __shared__ float s_a[tsize][tsize];
    __shared__ float s_b[tsize][tsize];

    for(int i=0;i < m/tsize; i++){
        s_a[threadIdx.y][threadIdx.x] = a[row*m + threadIdx.x + i*tsize];
        s_b[threadIdx.y][threadIdx.x] = b[m*(threadIdx.y+i*tsize)+col];
        __syncthreads();
        for(int p=0;p<tsize;p++){
            sum += s_a[threadIdx.y][p]*s_b[p][threadIdx.x];
        }
        __syncthreads();
    }
    if (col < m && row < m) c[m*row + col] = sum;
}

void checkMultiplicationCorrect(float *p1, float *p2, float *p3, int m){
    for (int i=0; i<m; i++){
        for (int j=0; j<m; j++){
            float val = 0;
            for (int k=0; k<m; k++){
                val += p1[i*m+k] * p2[k*m+j];
            }
            // check for being close enough
            if (abs(p3[i*m+j] - val) > 0.0001){
                printf("\nwrong multiplication\n");
                return;
            }
        }
    }
    printf("\ncorrect multiplication\n");
}


int main(){
    int m = matsize;
    size_t size = m * m * sizeof(float);

    //allocate memory for host
    float *p1 = (float *)malloc(size);
    float *p2 = (float *)malloc(size);
    float *p3 = (float *)malloc(size);

    //initialize host memory
    for (int i=0; i<m*m; i++){
        p1[i] = (float)rand()/RAND_MAX;
        p2[i] = (float)rand()/RAND_MAX;
    }
    float *dev1, *dev2, *dev3;
    cudaMalloc(&dev1, size);
    cudaMalloc(&dev2, size);
    cudaMalloc(&dev3, size);

    // copy data to device from host
    cudaMemcpy(dev1, p1, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dev2, p2, size, cudaMemcpyHostToDevice);

    //launch kernel
    int thr = tsize;
    int blc = m / thr;
    dim3 threads = dim3(thr, thr);
    dim3 blocks = dim3(blc, blc);
    // get the time of multiplication
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);
    multiply<<<blocks, threads>>>(dev1, dev2, dev3, m);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float time = 0;
    cudaEventElapsedTime(&time, start, stop);
    // copy back
    cudaMemcpy(p3, dev3, size, cudaMemcpyDeviceToHost);

    printf("time of multiplication with gpu: %f\n", time/1000);
    // checkMultiplicationCorrect(p1, p2, p3, m);
    //free memory
    cudaFree(dev1);
    cudaFree(dev2);
    cudaFree(dev3);
    free(p1);
    free(p2);
    free(p3);
}

time of multiplication with gpu: 2.585407



Here too. Using the same thread size and matrix size as before, we saw that it takes noticeably less time doing the same task by using tiling.

# Part 5 (بخش ه)

Even though we look at the matrices as 2d objects how they are stored in the memory is 1d. So accessing by rows and columns has a crucial impact on the total time required to execute tasks such as matrix multiplication.
</br>
reading by rows is reading continuously rather than reading separate points in memory that are some thing aroung one row (1024 or 10240) far apart.
</br>
When working with the first matrix, we read by rows so we have been using memory coalescing there. Therefore, as a way of making the multiplication faster we can thing of using the second matrix in the same way. What we have to do is to transpose it first and then use the same kind of indexing we acquired for the first matrix.
</br>
The problem is that transposing has a time cost itself. In the following code, I have implemented transposing by gpu and then multiplication. We see that as the transposing is time consuming, the overall result is not that much better.

with shared memory

In [60]:
%%cu
#include <stdio.h>
#include <stdlib.h>
// sizes for shared memory
#define trsize 4
#define matsize 1024

// kernel for transposing matrix
__global__ void transpose(float *a, float* c){
    // row and column for the current thread
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    c[col*matsize + row] = a[row*matsize + col];
}

//CUDA kernel for multiplication
__global__ void multiply(const float *a, const float *b, float *c, int m){
     // row and column for the current thread
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    // shared memory
    __shared__ float s_a[trsize][matsize];
    __shared__ float s_b[trsize][matsize];

    //load data to shared memory
    for (int i=0; i<m/trsize; i++){
        s_a[threadIdx.y][threadIdx.x + i*trsize] = a[row*m + threadIdx.x + i*trsize];
        s_b[threadIdx.x][threadIdx.y + i*trsize] = b[threadIdx.y + i*trsize+ m*col];
    }
    // making sure the data is loaded
    __syncthreads();

    float tmp = 0.0f;
    for (int k=0;k<m;k++){
        tmp += s_a[threadIdx.y][k] * s_b[threadIdx.x][k]; // dot product
    }
    if (col < m && row < m) c[m*row + col] = tmp; // assigning the result
}


void checkMultiplicationCorrect(float *p1, float *p2, float *p3, int m){
    for (int i=0; i<m; i++){
        for (int j=0; j<m; j++){
            float val = 0;
            for (int k=0; k<m; k++){
                val += p1[i*m+k] * p2[k*m+j];
            }
            // check for being close enough
            if (abs(p3[i*m+j] - val) > 0.0001){
                printf("\nwrong multiplication\n");
                return;
            }
        }
    }
    printf("\ncorrect multiplication\n");
}

int main(){
    int m = matsize;
    size_t size = m * m * sizeof(float);

    //allocate memory for host
    float *p1 = (float *)malloc(size);
    float *p2 = (float *)malloc(size);
    float *p3 = (float *)malloc(size);

    //initialize host memory
    for (int i=0; i<m*m; i++){
        p1[i] = (float)rand()/RAND_MAX;
        p2[i] = (float)rand()/RAND_MAX;
    }

    float *dp1, *dp2, *dp3, *dp4;
    cudaMalloc(&dp1, size);
    cudaMalloc(&dp2, size);
    cudaMalloc(&dp3, size);
    cudaMalloc(&dp4, size);

    // copy data to device from host
    cudaMemcpy(dp1, p1, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dp2, p2, size, cudaMemcpyHostToDevice);

    //launch kernel
    int thr = trsize;
    int blc = m / thr;
    dim3 threads = dim3(thr, thr);
    dim3 blocks = dim3(blc, blc);
    // get the time of multiplication
    cudaEvent_t begin, end;
    cudaEventCreate(&begin);
    cudaEventCreate(&end);
    cudaEventRecord(begin);
    transpose<<<blocks, threads>>>(dp2, dp4);
    multiply<<<blocks, threads>>>(dp1, dp4, dp3, m);
    cudaEventRecord(end);
    cudaEventSynchronize(end);
    float time = 0;
    cudaEventElapsedTime(&time, begin, end);
    // copy back
    cudaMemcpy(p3, dp3, size, cudaMemcpyDeviceToHost);

    printf("Matrix Multiplication size %d time %f\n",m, time/1000);
    checkMultiplicationCorrect(p1, p2, p3, m);
    //free memory
    cudaFree(dp1);
    cudaFree(dp2);
    cudaFree(dp3);
    free(p1);
    free(p2);
    free(p3);
}

Matrix Multiplication size 1024 time 0.055191

correct multiplication



without shared memory

In [62]:
%%cu
#include <stdio.h>
#include <stdlib.h>
// sizes for shared memory
#define trsize 4
#define matsize 1024

// kernel for transposing matrix
__global__ void transpose(float *a, float* c){
    // row and column for the current thread
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    c[col*matsize + row] = a[row*matsize + col];
}

//CUDA kernel for multiplication
__global__ void multiply(const float *a, const float *b, float *c, int m){
    // defining row and column based on current thread
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if(row < m && col < m){ // check for being inside the bounds
        float sum = 0;
        for (int i=0; i<m; i++){
            sum+= a[row*m+i] * b[i+m*col]; //dot product
        }
        c[row*m+col] = sum; //assign the value to the result
    }
}

void checkMultiplicationCorrect(float *p1, float *p2, float *p3, int m){
    for (int i=0; i<m; i++){
        for (int j=0; j<m; j++){
            float val = 0;
            for (int k=0; k<m; k++){
                val += p1[i*m+k] * p2[k*m+j];
            }
            // check for being close enough
            if (abs(p3[i*m+j] - val) > 0.0001){
                printf("\nwrong multiplication\n");
                return;
            }
        }
    }
    printf("\ncorrect multiplication\n");
}

int main(){
    int m = matsize;
    size_t size = m * m * sizeof(float);

    //allocate memory for host
    float *p1 = (float *)malloc(size);
    float *p2 = (float *)malloc(size);
    float *p3 = (float *)malloc(size);

    //initialize host memory
    for (int i=0; i<m*m; i++){
        p1[i] = (float)rand()/RAND_MAX;
        p2[i] = (float)rand()/RAND_MAX;
    }

    float *dp1, *dp2, *dp3, *dp4;
    cudaMalloc(&dp1, size);
    cudaMalloc(&dp2, size);
    cudaMalloc(&dp3, size);
    cudaMalloc(&dp4, size);

    // copy data to device from host
    cudaMemcpy(dp1, p1, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dp2, p2, size, cudaMemcpyHostToDevice);

    //launch kernel
    int thr = trsize;
    int blc = m / thr;
    dim3 threads = dim3(thr, thr);
    dim3 blocks = dim3(blc, blc);
    // get the time of multiplication
    cudaEvent_t begin, end;
    cudaEventCreate(&begin);
    cudaEventCreate(&end);
    cudaEventRecord(begin);
    transpose<<<blocks, threads>>>(dp2, dp4);
    multiply<<<blocks, threads>>>(dp1, dp4, dp3, m);
    cudaEventRecord(end);
    cudaEventSynchronize(end);
    float time = 0;
    cudaEventElapsedTime(&time, begin, end);
    // copy back
    cudaMemcpy(p3, dp3, size, cudaMemcpyDeviceToHost);

    printf("Matrix Multiplication size %d time %f\n",m, time/1000);
    checkMultiplicationCorrect(p1, p2, p3, m);
    //free memory
    cudaFree(dp1);
    cudaFree(dp2);
    cudaFree(dp3);
    free(p1);
    free(p2);
    free(p3);
}

Matrix Multiplication size 1024 time 0.046143

correct multiplication



we see that in this case, not using shared memory is a bit faster.